### This notebook was used to create JRA55 daily dataset

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import aostools.climate as aos
import seaborn as sns
import numpy as np

In [2]:
# Pull in large dataset and manipulate to smaller one

ds = xr.open_mfdataset('/disca/share/sit204/jra_55/1958_2016/atmos_daily_uvtw.nc', 
                       parallel=True, chunks={'time': 31})

# maybe need: , chunks={'time': 31}

# rename variables
rename_variables = {'var33': 'u',
                    'var34': 'v',
                    'var11': 't',
                    'var39': 'omega'}

ds = ds.drop_dims('bnds')
ds.attrs = {}

# manipulate ds further
ds['lev'] = ds.lev.values / 100
ds['lev_2'] = ds.lev_2.values / 100

ds = ds.rename(rename_variables)
ds 

# # extract DJF months from dataset
# import datetime as dt

# # ds = ds.where(ds.time.dt.month.isin([12, 1, 2]))
# ds = ds.sel(time=ds.time.dt.month.isin([12, 1, 2]))
# ds.to_netcdf('jra55_djf.nc')

<xarray.Dataset>
Dimensions:  (time: 21550, lon: 144, lat: 73, lev: 37, lev_2: 37)
Coordinates:
  * time     (time) datetime64[ns] 1958-01-01T09:00:00 ... 2016-12-31T09:00:00
  * lon      (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * lat      (lat) float64 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * lev      (lev) float64 1e+03 975.0 950.0 925.0 900.0 ... 7.0 5.0 3.0 2.0 1.0
  * lev_2    (lev_2) float64 1.0 2.0 3.0 5.0 7.0 ... 925.0 950.0 975.0 1e+03
Data variables:
    u        (time, lev, lat, lon) float32 dask.array<chunksize=(31, 37, 73, 144), meta=np.ndarray>
    v        (time, lev_2, lat, lon) float32 dask.array<chunksize=(31, 37, 73, 144), meta=np.ndarray>
    t        (time, lev_2, lat, lon) float32 dask.array<chunksize=(31, 37, 73, 144), meta=np.ndarray>
    omega    (time, lev_2, lat, lon) float32 dask.array<chunksize=(31, 37, 73, 144), meta=np.ndarray>

In [3]:
# Import new, smaller dataset
# ds = xr.open_dataset('/home/links/ct715/reanalysis_data/eddy_feedback/daily_datasets/datasets/JRA55_DJF/jra55_djf.nc')

In [4]:
# # compute zonal mean zonal wind
# ubar = ds.u.mean(('time','lon'))
# ds

# Need to change dimension on zonal wind

In [5]:
# ubar.plot.contourf(levels=20, yincrease=False, yscale='linear', figsize=(8,4))

In [6]:
ds['ucomp_flip'] = (ds['v'].dims, ds['u'][:,::-1,:,:].values)

# ds['ucomp_flip'].mean(('time', 'lon')).plot.contourf(levels=20, yincrease=False)

In [7]:
# check dims are definitely same but flipped
# ds.lev[::-1].values - ds.lev_2.values

In [8]:
ds = ds.drop_vars('u')
ds = ds.drop_dims('lev')

rename_variables = {'lev_2': 'level', 'ucomp_flip': 'u'}
ds = ds.rename(rename_variables)
ds

<xarray.Dataset>
Dimensions:  (time: 21550, lon: 144, lat: 73, level: 37)
Coordinates:
  * time     (time) datetime64[ns] 1958-01-01T09:00:00 ... 2016-12-31T09:00:00
  * lon      (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * lat      (lat) float64 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * level    (level) float64 1.0 2.0 3.0 5.0 7.0 ... 925.0 950.0 975.0 1e+03
Data variables:
    v        (time, level, lat, lon) float32 dask.array<chunksize=(31, 37, 73, 144), meta=np.ndarray>
    t        (time, level, lat, lon) float32 dask.array<chunksize=(31, 37, 73, 144), meta=np.ndarray>
    omega    (time, level, lat, lon) float32 dask.array<chunksize=(31, 37, 73, 144), meta=np.ndarray>
    u        (time, level, lat, lon) float32 -56.05 -55.99 ... -2.199 -1.996

# Check with some plots

In [9]:
import sys
sys.path.append('/home/links/ct715/eddy_feedback/')

import functions.eddy_feedback as ef 

In [11]:
ds = ef.calculate_epfluxes_ubar(ds)
ds

<xarray.Dataset>
Dimensions:  (time: 21550, lon: 144, lat: 73, level: 37)
Coordinates:
  * time     (time) datetime64[ns] 1958-01-01T09:00:00 ... 2016-12-31T09:00:00
  * lon      (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * lat      (lat) float64 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * level    (level) float64 1.0 2.0 3.0 5.0 7.0 ... 925.0 950.0 975.0 1e+03
Data variables:
    v        (time, level, lat, lon) float32 dask.array<chunksize=(31, 37, 73, 144), meta=np.ndarray>
    t        (time, level, lat, lon) float32 dask.array<chunksize=(31, 37, 73, 144), meta=np.ndarray>
    omega    (time, level, lat, lon) float32 dask.array<chunksize=(31, 37, 73, 144), meta=np.ndarray>
    u        (time, level, lat, lon) float32 -56.05 -55.99 ... -2.199 -1.996
    ubar     (level, lat) float32 6.883e-06 2.597 4.762 ... -2.494 9.035e-06
    ep1      (time, level, lat) float64 0.005775 -9.279 ... 2.268 0.0006846
    ep2      (time, level, lat) float64 -4.366e-12 -0.0002097 ... -0.01421 -0.0
    div1     (time, level, lat) float64 -2.556e+12 8.195 ... 0.5563 3.03e+11
    div2     (time, level, lat) float64 5.635e-07 4.18 ... -1.299 9.565e-07

In [14]:
ds.to_netcdf('jra55_uvtw_ep.nc')